In [1]:
# import packages
import matplotlib.pyplot as plt
from astropy.timeseries import LombScargle
import numpy as np
import pandas as pd

"""
IMPORTANT:
For this program to work the researcher needs to put both csv files
cycle1.csv and cycle2.csv in a folder called Observations. Run
DownloadMongoDB.ipynb to get the files. You will have to manually
put these new files in the Observations folder which will be located
in the same folder as this file.
"""

"""
Headers for Cycle1 DF (Northern Hemisphere):
_id, Original_Object_ID, MDJ, Magnitude, Magnitude_Error

Cycle1 Size: 28878 Asteroids

Headers for Cycle2 DF (Southern Hemisphere):
_id, Object_ID, MDJ, Magnitude, Magnitude_Error, Sector

Cycle2 Size: 18712 Asteroids

Overlap Size: 3025 Asteroids
"""

# import data from csv file into a pandas df
cycle1_df = pd.read_csv("Observations/cycle1.csv", low_memory=False)
cycle2_df = pd.read_csv("Observations/cycle2.csv", low_memory=False)

# get the ids that intersect
cycle1_df_unique = np.unique(cycle1_df["Original_Object_ID"])
cycle2_df_unique = np.unique(cycle2_df["Object_ID"])
intersect_ids_df = np.intersect1d(cycle1_df_unique, cycle2_df_unique)

In [ ]:
"""
SUPPORTING FUNCTIONS
"""

# calculates the chi2 value based on the observed, expected, and error values
def reduced_chi_square(observed, expected, error):

    chi2 = 0

    for observation in range(0, len(observed)):
        chi2 += ((observed[observation] - expected[observation]) ** 2) / error[observation]

    chi2 = chi2 / len(observed - 4)

    return chi2

# compares cycle1 and cycle2 and compares the amplitude and period of the sine curve
def compare_cycles(cycle1_df, cycle2_df, intersect_ids):

    columns = ["asteroid_id", "period_(hr)_1", "amplitude_1",    
               "period_(hr)_2", "amplitude_2"]
    
    results_df = pd.DataFrame(columns=columns)

    intersect_ids_df = pd.DataFrame(intersect_ids, columns=["asteroid_id"])

    for row in intersect_ids_df.itertuples(index=False):
        asteroid_id = row.asteroid_id

        row1 = cycle1_df[cycle1_df["asteroid_id"] == asteroid_id]
        row2 = cycle2_df[cycle2_df["asteroid_id"] == asteroid_id]

        if row1.empty or row2.empty:
            continue

        period1 = row1["period_(hr)"].values[0]
        amplitude1 = row1["amplitude"].values[0]
        period2 = row2["period_(hr)"].values[0]
        amplitude2 = row2["amplitude"].values[0]

        results_df.loc[len(results_df)] = [
            asteroid_id,
            period1,
            amplitude1,
            period2,
            amplitude2
        ]

    return results_df

# defines the sine function to fit on the light curve plot
def sine_function(x, amplitude, frequency, phase, offset):
    
    return amplitude * np.sin(frequency * x + phase) + offset

In [ ]:
"""
MAIN SUPPORTING FUNCTION:
Derive the rotational period of an object from MDJ
and magnitude to get a periodogram. With this take the
max power to create a sine curve
"""

def find_asteroid_light_curve(asteroid, cycle_num, object_id):
        
        # extract MJD, magnitude, magnitude error
        time_mdj = asteroid["MJD"].values
        magnitude = asteroid["Magnitude"].values
        error = asteroid["Magnitude_Error"].values

        # define frequency range (1 to 24 hours period = 1/24 to 1 cycles/hour)
        # convert MJD to hours relative to first timestamp
        time_hours = (time_mdj - time_mdj.min()) * 24
        frequency = np.linspace(1/(max(time_hours) - min(time_hours)), 1/2, 1000000)
        power = LombScargle(time_hours, magnitude, error).power(frequency)

        # best period is 1 / frequency at peak power
        peak_index = np.argmax(power)
        peak_power = power[peak_index]
        best_frequency = frequency[peak_index]
        best_period = 1 / best_frequency

        # double the period for full rotation
        best_period = best_period * 2

        # compute sine curve fit
        time_fit_hours = np.linspace(0, best_period, 1000)
        ls = LombScargle(time_hours, magnitude, error)
        magnitude_fit_smoothed = ls.model(time_fit_hours, best_frequency)
        magnitude_fit = ls.model(time_hours, best_frequency)

        # get data to scrape
        amplitude = max(magnitude_fit_smoothed) - min(magnitude_fit_smoothed)
        reduced_chi2_statistic = reduced_chi_square(magnitude, magnitude_fit, error)

        """
        Plot the periodogram and the lomb scargle model for testing
        """

        # plot the periodogram
        # plt.plot(frequency, power)
        # plt.show()

        # plot the lomb scargle model need to add the magnitude fit
        # plt.errorbar(time_hours % best_period, magnitude, yerr = error, fmt = "o", ecolor = "red", 
        #              color = "blue", elinewidth = 1, capsize = 2, alpha = 0.5)
        # plt.plot(time_fit_hours, magnitude_fit_smoothed, color = "green", linewidth = 3)
        # plt.title(f"Light Curve Phase for: {object_id}")
        # plt.xlabel("Hours")
        # plt.ylabel("Magnitude")
        # plt.gca().invert_yaxis()
        # plt.grid(True)
        # plt.show()

        """
        Plotting the raw time series data for testing
        """

        # plot the timeseries data
        # plt.errorbar(asteroid["MJD"], asteroid["Magnitude"], yerr = asteroid["Magnitude_Error"], 
        #              fmt = "o", ecolor = "red", color = "orange", elinewidth = 1, capsize = 2)
        # plt.title(f"Magnitude vs MJD for Object ID {object_id}")
        # plt.xlabel("MJD")
        # plt.ylabel("Magnitude")
        # plt.gca().invert_yaxis()
        # plt.grid(True)
        # plt.show()

        return [cycle_num, object_id, reduced_chi2_statistic, 
                peak_power, best_period, amplitude, len(time_mdj)]


def find_cycle_asteroids_light_curve_fit(cycle_df, id_col_name, cycle_num):

    # initialize output df
    columns = ["cycle", "asteroid_id", "reduced_chi2", "peak_power", 
               "period_(hr)", "amplitude", "num_observations"]
    
    results_df = pd.DataFrame(columns=columns)

    # group each asteroid by their object id
    asteroids = cycle_df.groupby(id_col_name)

    count = 0

    # plot each asteroid
    for object_id, asteroid in asteroids:

        result = find_asteroid_light_curve(asteroid, cycle_num, object_id)

        # add data to the results dataframe
        results_df.loc[len(results_df)] = result

        # limits the number of asteroids to 10 computed for testing
        count += 1
        if count >= 10:
            break

    return results_df

In [4]:
# compute all results
cycle1_results_df = find_cycle_asteroids_light_curve_fit(cycle1_df, "Original_Object_ID", 1)
cycle2_results_df = find_cycle_asteroids_light_curve_fit(cycle2_df, "Object_ID", 2)
combine_results_df = compare_cycles(cycle1_results_df, cycle2_results_df, intersect_ids_df)

# convert dataframes to csv
cycle1_results_df.to_csv('cycle1_results.csv', index=False)
cycle2_results_df.to_csv('cycle2_results.csv', index=False)
combine_results_df.to_csv('combine_results.csv', index=False)